# Helper Functions

In [4]:
def parse_p_median(file, target_id, target_best_value):
    with open(file,"r") as f:
        lines = [line.strip() for line in f if line.strip()]
    i=0
    while i<len(lines):
        parts=lines[i].split()
        if len(parts)==2:
            current_id=int(parts[0])
            current_best_value=int(parts[1])
        if current_id==target_id and current_best_value==target_best_value:
            constraints=lines[i+1].split()
            number_of_customers=int(constraints[0])
            number_of_medians=int(constraints[1])
            capacity_of_medians=int(constraints[2])
            customers=[]
            for j in range(i+2,i+2+number_of_customers):
                each_line=lines[j].split()
                customers.append({
                    "id": int(each_line[0]),
                    "x_coordinate":float(each_line[1]),
                    "y_coordinate":float(each_line[2]),
                    "demand":int(each_line[3])
                })
            return {
                "number_of_medians":number_of_medians,
                "capacity_of_medians":capacity_of_medians,
                "customers":customers
            }
        i+=1
    return None
print(parse_p_median("p_median_capacitated.txt",1,713))
        

{'number_of_medians': 5, 'capacity_of_medians': 120, 'customers': [{'id': 1, 'x_coordinate': 2.0, 'y_coordinate': 62.0, 'demand': 3}, {'id': 2, 'x_coordinate': 80.0, 'y_coordinate': 25.0, 'demand': 14}, {'id': 3, 'x_coordinate': 36.0, 'y_coordinate': 88.0, 'demand': 1}, {'id': 4, 'x_coordinate': 57.0, 'y_coordinate': 23.0, 'demand': 14}, {'id': 5, 'x_coordinate': 33.0, 'y_coordinate': 17.0, 'demand': 19}, {'id': 6, 'x_coordinate': 76.0, 'y_coordinate': 43.0, 'demand': 2}, {'id': 7, 'x_coordinate': 77.0, 'y_coordinate': 85.0, 'demand': 14}, {'id': 8, 'x_coordinate': 94.0, 'y_coordinate': 6.0, 'demand': 6}, {'id': 9, 'x_coordinate': 89.0, 'y_coordinate': 11.0, 'demand': 7}, {'id': 10, 'x_coordinate': 59.0, 'y_coordinate': 72.0, 'demand': 6}, {'id': 11, 'x_coordinate': 39.0, 'y_coordinate': 82.0, 'demand': 10}, {'id': 12, 'x_coordinate': 87.0, 'y_coordinate': 24.0, 'demand': 18}, {'id': 13, 'x_coordinate': 44.0, 'y_coordinate': 76.0, 'demand': 3}, {'id': 14, 'x_coordinate': 2.0, 'y_coordi

# SPEA Algorithm

In [ ]:
#Constraints- we have a fixed number of facilities to place and each facility has a maximum capacity
#Objective A- minimize facility distance(we want them to be as close to their respective customers as possible)
#Objective B- we want the facilities to be spread far apart from each other